In [ ]:
def run_cmd(ip,cmd)
  puts "====begin cmd #{time=Time.now} @#{ip}===="
  cmd = "ssh -i ~/.ssh/LightsailDefaultKey-us-east-1.pem -o 'StrictHostKeyChecking no' ubuntu@#{ip} '#{cmd}'"
  puts cmd
  system(cmd)
  puts "====end cmd #{Time.now} @#{ip} time #{Time.now-time} s===="
end

require 'parallel'

def worker_run(worker,public_ips,cmd)
  puts "====worker_run begin cmd #{time=Time.now}===="
  Parallel.map(worker,in_threads: 10) { |w| run_cmd(public_ips[w],cmd) }
  puts "====worker_run end cmd #{Time.now} time #{Time.now-time} s===="
end

def worker_run_script(worker,public_ips,script)
  cmd = script.split("\n")
  cmd.map {|c|
    worker_run(worker,public_ips,c)
  }
end



aws lightsail create-instances --no-cli-pager --instance-names {'panworker-1'} --availability-zone 'us-east-1a' --blueprint-id 'ubuntu_20_04' --bundle-id 'medium_2_0'


false

In [ ]:
workers = 1

worker = (1..workers).map {|x| "panworker-#{x}"}

if worker.size>1 then
  create_worker = "aws lightsail create-instances --no-cli-pager --instance-names {#{worker.map{|x| "'#{x}'"}.join(',')}} --availability-zone 'us-east-1a' --blueprint-id 'ubuntu_20_04' --bundle-id 'medium_2_0'"
else
  create_worker = "aws lightsail create-instances --no-cli-pager --instance-names #{worker.map{|x| "'#{x}'"}.join(',')} --availability-zone 'us-east-1a' --blueprint-id 'ubuntu_20_04' --bundle-id 'medium_2_0'"
end
puts create_worker
system(create_worker)

In [ ]:
require 'JSON'

get_public_ip = "aws lightsail get-instances --no-cli-pager --region 'us-east-1' --query 'instances[].{name:name,publicIpAddress:publicIpAddress}'"
puts get_public_ip
data = `#{get_public_ip}`
public_ips = JSON.parse(data).map {|x| [x["name"],x["publicIpAddress"]]}.to_h

In [ ]:
script = <<~CODE
sudo apt-get update
sudo apt-get install -y ca-certificates curl gnupg lsb-release
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update
sudo apt-get install -y docker-ce docker-ce-cli containerd.io
sudo usermod -aG docker $USER
newgrp docker 
sudo systemctl enable docker.service
sudo systemctl enable containerd.service
CODE

worker_run_script(worker,public_ips,script)
worker_run_script(worker,public_ips,"docker run hello-world")


In [ ]:
script = <<~CODE
docker container run -d --restart=always -e DB_CONNECT_STR='__PARAMS_CONNECT_STR__' -e WORKER_NAME='__WORKER__' --name __WORKER__  adam429/pan-repo:panworker
CODE

require 'securerandom'

output = Parallel.map(worker,in_threads: 1) { |w| 
  script_a = script.gsub(/__WORKER__/,"#{w}_#{SecureRandom.hex(2)}").gsub(/__PARAMS_CONNECT_STR__/,+ENV["DB_CONNECT_STR"])
  worker_run_script([w],public_ips,script_a)

  script_a = script.gsub(/__WORKER__/,"#{w}_#{SecureRandom.hex(2)}").gsub(/__PARAMS_CONNECT_STR__/,+ENV["DB_CONNECT_STR"])
  worker_run_script([w],public_ips,script_a)
}



In [ ]:
# delete workers

Parallel.map(worker,in_threads: 10) { |w| 
  delete_worker = "aws lightsail delete-instance --no-cli-pager --instance-name #{w}" 
  puts delete_worker
  system(delete_worker)
}


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>